# Parallel Computing

## Processors and threads

Every Julia session has some number of *processors* (which do not share memory), and each processor has some number of *threads* (which do share memory).

In [ ]:
nprocs()

Processes are distinct from threads:

In [ ]:
Threads.nthreads()

The Julia process connected to your keyboard is processor 1 (sometimes called the "master" or "head node").

`addprocs(n)` starts additional processors on your local machine, as separate OS-level processes. Processors other than 1 are sometimes called "workers".

In [ ]:
myid()

In [ ]:
addprocs(2)

In [ ]:
nprocs()

In [ ]:
nworkers()

In [ ]:
workers()

## Tasks

Processors and threads represent compute resources. *Tasks* represent units of work performed by those resources.

A `Task` gives you a handle to a computation to be performed. It has a create-start-run-finish lifecycle:

* Create: `Task(thunk)` or `@task expr`. The latter is equivalent to `Task(()->expr)`.
* Start: `schedule(task)`. `@async` and `@schedule` do create+start in one step.
* Run: `current_task()` is always running. A task cooperatively pauses itself using `wait` or `yield`.
* Finish: the task's `thunk` must exit via return or exception. You can `wait(task)`.

Technically, `Task` is a *concurrency* primitve (as opposed to a *parallelism* primitive).

In [ ]:
work = @task sum(rand(1000))

In [ ]:
schedule(work)

In [ ]:
wait(work)

In [ ]:
@time work = @schedule sum(rand(100000000))

In [ ]:
wait(work)

In [ ]:
@time sum(rand(100000000))

In [ ]:
@sync for t = 'A':'F'
    @async for k = 1:10
        println("$t$k @ $(now())")
        sleep(rand())
    end
end

## Communication: Conditions and Channels

Most of the time, Tasks are too low-level to use directly.

* A `Condition` represents an edge-triggered event that Tasks can `wait` for and `notify`.
* I/O operations use Conditions, wait, and notify internally.
* Parallel primitives start and manage Tasks internally.

In [ ]:
event = Condition()

In [ ]:
work = @schedule begin
    println("I'm waiting for the event to happen.")
    wait(event)
    println("Ok, it happened.")
end

In [ ]:
notify(event)

A `Channel` is similar, but adds the ability to send data.

In [ ]:
c = Channel(4)  # 4 is the queue size

In [ ]:
printer = @schedule begin
    while true
        data = take!(c)
        println("from background: ", data)
    end
end

In [ ]:
put!(c, rand(2,2));

In [ ]:
put!(c, 1);

In [ ]:
put!(c, "hello");

A channel needs to be `close`d eventually.

## Putting it together: remote execution

`addprocs` sets up Tasks on all processors ready to run code for you.

A single remote function call can be done with `remotecall`, which returns a `Future`:

In [ ]:
futuremat = remotecall(rand, 2, 3, 3)

In [ ]:
fetch(futuremat)

In [ ]:
isready(futuremat)

In [ ]:
@schedule begin
    wait(futuremat)
    println("got it, callback goes here")
    # ......
end

`fetch` should be avoided, because it moves data. However, it's efficient when used on the process that owns the data.

`@spawnat` rewrites to `remotecall`, similar to `@task` for `Task` etc.

In [ ]:
futuremat_plus_1 = @spawnat 2 1+fetch(futuremat)

In [ ]:
fetch(futuremat_plus_1)

`RemoteChannel` works similarly to `Channel`, but can be used by any process.

In [ ]:
rc = RemoteChannel()

In [ ]:
@spawnat 2 begin
    @show rc
    while true
        val = take!(rc)
        println(val)
    end
end

In [ ]:
put!(rc, "test")

In [ ]:
put!(rc, rand(2,2))

## Spawn and sync

* `@spawn` is like `@spawnat` but picks a processor for you
* `@sync` waits for all dynamically-enclosing `@spawn`, `@spawnat`, and `@async`

#### Example

Recipe for parallel-loading many files from a shared file system:
```jl
dataset = [(@spawn load(file)) for file in files]
results = [(@spawnat x.where f(fetch(x))) for x in dataset]
```

In [ ]:
futuremat_plus_1.where

## Loading code

Use this recipe:

1. `using Thing1, Thing2` for packages used only on process 1
2. `addprocs()`
3. `using Thing3` for packages needed everywhere
4. Any time: `include(...)` is always local-only; `@everywhere include(...)` for global

**Note:** if you need to send an object of type T to another processor, the package defining T must be loaded there.